## Download Data


In [0]:
# Authorization In Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
from google.colab import drive

zip_ref = zipfile.ZipFile("/content/drive/My Drive/right.zip", 'r')
zip_ref.extractall("/content/drive/My Drive/train_r")
zip_ref.close()

# zip_ref = zipfile.ZipFile("/content/drive/My Drive/left_small_val.zip", 'r')
# zip_ref.extractall("/tmp/Dataset")
# zip_ref.close()

KeyboardInterrupt: ignored

In [0]:
import os
path_train = "/content/drive/My Drive/train"

path_val = "/content/drive/My Drive/val"

# Directory with our training horse pictures
train_first_dir = os.path.join('/tmp/Dataset/left_train/1')

# Directory with our training human pictures
train_second_dir = os.path.join('/tmp/Dataset/left_train/2')

train_third_dir = os.path.join('/tmp/Dataset/left_train/3')


# Directory with our training horse pictures
validation_first_dir = os.path.join(path_val + '1')

# Directory with our training human pictures
validation_second_dir = os.path.join(path_val + '2')

validation_third_dir = os.path.join(path_val + '3')

In [0]:
# Total Number Images In Directories
print('total training first images:', len(os.listdir(train_first_dir)))
print('total training second images:', len(os.listdir(train_second_dir)))
print('total training third images:', len(os.listdir(train_third_dir)))

total training first images: 404
total training second images: 317
total training third images: 298


## Build Model

In [0]:
import tensorflow as tf

number_classes = 8
input_shape = (245, 310, 1)

model = tf.keras.models.Sequential([
    # 5 Conv Layers With MaxPolling 

    tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(8, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(3,3),
    

    # Flatten Layer In One Array
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(number_classes, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 243, 308, 8)       80        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 241, 306, 8)       584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 80, 102, 8)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 65280)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 522248    
Total params: 522,912
Trainable params: 522,912
Non-trainable params: 0
_________________________________________________________________


### Compile Model

In [0]:
from tensorflow.keras.optimizers import RMSprop


model.compile(optimizer = RMSprop(lr=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

## Data Generation

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_size = (245, 310)

rotation = 15
import cv2
import numpy as np
from PIL import Image


def myFunc(image):
  image = np.array(image)
  hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  return Image.fromarray(hsv_image)



# Normalization Data 
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255, rotation_range=rotation, zca_whitening=True)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/train_r",  # This is the source directory for training images
        target_size=target_size,  # All images will be resized to 245, 310
        batch_size=120,
        color_mode="grayscale",
        class_mode="categorical")

# Validation Generator 
validation_datagen = ImageDataGenerator(rescale=1/255, rotation_range=rotation)
validation_generator = validation_datagen.flow_from_directory(
    "/content/drive/My Drive/val_r",
    target_size=target_size,
    batch_size=45,
    color_mode="grayscale",
    class_mode="categorical")



/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 1528 images belonging to 8 classes.
Found 200 images belonging to 8 classes.



### Training

In [0]:
# Steps Per Epochs
history = model.fit_generator(
      train_generator,
      steps_per_epoch=10,
      epochs=25,
      validation_data=validation_generator,
      validation_steps=4,
      verbose=2
      )

Epoch 1/25


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
10/10 - 45s - loss: 1.1341 - acc: 0.6592 - val_loss: 1.1520 - val_acc: 0.6444
Epoch 2/25
Epoch 1/25
10/10 - 8s - loss: 1.0166 - acc: 0.6910 - val_loss: 0.8203 - val_acc: 0.7611
Epoch 3/25
Epoch 1/25
10/10 - 9s - loss: 0.7274 - acc: 0.8175 - val_loss: 0.6565 - val_acc: 0.8222
Epoch 4/25
Epoch 1/25
10/10 - 8s - loss: 0.6890 - acc: 0.8356 - val_loss: 0.6682 - val_acc: 0.7556
Epoch 5/25
Epoch 1/25
10/10 - 9s - loss: 0.5499 - acc: 0.8692 - val_loss: 0.6742 - val_acc: 0.7889
Epoch 6/25
Epoch 1/25
10/10 - 8s - loss: 0.5043 - acc: 0.8952 - val_loss: 0.6280 - val_acc: 0.8111
Epoch 7/25
Epoch 1/25
10/10 - 8s - loss: 0.3981 - acc: 0.9118 - val_loss: 0.4407 - val_acc: 0.8667
Epoch 8/25
Epoch 1/25
10/10 - 8s - loss: 0.3186 - acc: 0.9358 - val_loss: 0.2861 - val_acc: 0.9278
Epoch 9/25
Epoch 1/25
10/10 - 8s - loss: 0.3031 - acc: 0.9332 - val_loss: 0.3006 - val_acc: 0.9333
Epoch 10/25
Epoch 1/25
10/10 - 9s - loss: 0.2439 - acc: 0.9558 - val_loss: 0.2864 - val_acc: 0.9278
Epoch 11/25
Epoch 1

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: ignored

In [0]:
### Saving Model
model.save('/content/drive/My Drive/Comtech_First_Model_99_Right_8.h5')

In [0]:
import numpy as np
from keras.preprocessing import image
cnt = 0
for i in range(10,99):
  if os.path.exists("/content/drive/My Drive/val/1_r/{}".format("source1_0"+str(i)+".bmp")) and cnt < 20:
    img = image.load_img(("/content/drive/My Drive/val_r/1/{}".format("source1_0"+str(i)+".bmp")), target_size=target_size,color_mode='grayscale')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    print(model.predict(x))
    cnt += 1


In [0]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/drive/My Drive/train_r/1/source1_218.bmp'

img = image.load_img(img_path, target_size=target_size, color_mode="grayscale")
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

print(img_tensor.shape)
# These are the names of the layers, so can have them as part of our plot
from tensorflow.keras import models

# Extracts the outputs of the top 8 layers:
layer_outputs = [layer.output for layer in model.layers[:8]]
# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
print('Activation Model', activation_model)

# This will return a list of 5 Numpy arrays:
# one array per layer activation
activations = activation_model.predict(img_tensor)

# Here we can activated First Layers in NN
first_layer_activation = activations[0]
print(first_layer_activation.shape)
layer_names = []
for layer in model.layers[:5]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, activations):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size, size_2 = layer_activation.shape[1], layer_activation.shape[2]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row

    display_grid = np.zeros((size * n_cols, images_per_row * size_2))
    
    # (486, 4928) - 243 * 2, 308 * 16
    # print(display_grid.shape)
    
    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            
            display_grid[col * size : (col + 1) * size,
                         row * size_2 : (row + 1) * size_2] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()

(1, 245, 310, 1)
Activation Model <tensorflow.python.keras.engine.training.Model object at 0x7f2804af1cc0>
(1, 243, 308, 8)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.


IndexError: ignored

RuntimeError: ignored

<Figure size 1460.15x0 with 1 Axes>

RuntimeError: ignored

<Figure size 1462.71x0 with 1 Axes>

RuntimeError: ignored

<Figure size 1468.8x0 with 1 Axes>